In [1]:
#This code is to see if we can use the Wilcoxon test after using Boruta to see which groups are significant to rule out any other groups.
#I'll compare this to the boxplot results from the boruta run and see if it makes sense with its selection for significant values via wilcoxon test.
#I'll be doing the same thing with XGBoost to determine which is better.
library(dplyr)
library(tidyr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
df <- read.csv("Significant_Features_Preliminary_Cohort1_Heart_C8.csv")

In [3]:
#Sets feature columns, my first 5 were metadata. Adjust accordingly.
feature_cols <- names(df)[6:ncol(df)]  #Assuming first 5 columns are metadata

#Pivot to long format, this is to transpose everything
df_long <- df %>%
  pivot_longer(
    cols = all_of(feature_cols),
    names_to = "Feature",
    values_to = "Value"
  )

# Get all group names
all_groups <- unique(df_long$ATTRIBUTE_Group_number)

# Set reference groups
reference_groups <- c("Group1", "Group3")

# Initialize empty list to collect all results
all_results <- list()

In [4]:
#Loops everything by comparing each feature in each treatment group individually to either group1 or group3.
for (ref_group in reference_groups) {
  for (comp_group in setdiff(all_groups, ref_group)) {
    
    subset_data <- df_long %>%
      filter(ATTRIBUTE_Group_number %in% c(ref_group, comp_group))
    
    result <- subset_data %>%
      group_by(Feature) %>%
      summarise(
        p_value = tryCatch(
          wilcox.test(Value ~ ATTRIBUTE_Group_number)$p.value,
          error = function(e) NA
        ),
        .groups = "drop"
      ) %>%
      mutate(
        comparison = paste(ref_group, "vs", comp_group, sep = "_"),
        p_adj = p.adjust(p_value, method = "BH")
      )
    
    # Add result to list
    all_results[[paste(ref_group, comp_group, sep = "_")]] <- result
  }
}

# Combine all results into one data frame
final_results <- bind_rows(all_results)

# Save to one CSV
write.csv(final_results, "wilcoxon_all_results.csv", row.names = FALSE)

Warning message:
"There were 6 warnings in `summarise()`.
The first warning was:
ℹ In argument: `p_value = tryCatch(...)`.
ℹ In group 1: `Feature = "X153.0405_0.36_315"`.
Caused by warning in `wilcox.test.default()`:
! cannot compute exact p-value with ties
ℹ Run `dplyr::last_dplyr_warnings()` to see the 5 remaining warnings."
Warning message:
"There were 8 warnings in `summarise()`.
The first warning was:
ℹ In argument: `p_value = tryCatch(...)`.
ℹ In group 1: `Feature = "X153.0405_0.36_315"`.
Caused by warning in `wilcox.test.default()`:
! cannot compute exact p-value with ties
ℹ Run `dplyr::last_dplyr_warnings()` to see the 7 remaining warnings."
Warning message:
"There were 5 warnings in `summarise()`.
The first warning was:
ℹ In argument: `p_value = tryCatch(...)`.
ℹ In group 1: `Feature = "X153.0405_0.36_315"`.
Caused by warning in `wilcox.test.default()`:
! cannot compute exact p-value with ties
ℹ Run `dplyr::last_dplyr_warnings()` to see the 4 remaining warnings."
Warning messa